In [51]:
# imports
import os
import tensorflow as tf
import numpy as np
import keras
import pandas as pd 
import imageio.v2 as imageio
import cv2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [52]:
# Sets the current working directory to the path specified
path = 'C:\\Users\\Sivar\\GitHub\\Breast_Cancer_Image_Segmentation\\data'
os.chdir(path)

In [53]:

def preprocess_images_and_masks(folder_path):
  # Load the images and masks from the folder
  images = [cv2.imread(os.path.join(folder_path, f)) for f in os.listdir(folder_path) if f.endswith('_image.png')]
  masks = [cv2.imread(os.path.join(folder_path, f), cv2.IMREAD_GRAYSCALE) for f in os.listdir(folder_path) if f.endswith('_mask.png')]

  # Resize the images and masks to a desired size
  desired_size = (256, 256)
  images = [cv2.resize(image, desired_size) for image in images]
  masks = [cv2.resize(mask, desired_size, interpolation=cv2.INTER_NEAREST) for mask in masks]

  # Convert the images to RGB format (if they are not already)
  images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in images]

  # Normalize the pixel values of the images and masks
  images = [image / 255.0 for image in images]
  masks = [mask / 255.0 for mask in masks]

  # Convert the images and masks to numpy arrays
  images = np.array(images)
  masks = np.array(masks)

  return images, masks



In [54]:
# Define the paths to the three folders
benign_folder = 'C:\\Users\\Sivar\\GitHub\\Breast_Cancer_Image_Segmentation\\data\\benign'
malignant_folder = 'C:\\Users\\Sivar\\GitHub\\Breast_Cancer_Image_Segmentation\\data\\malignant'
normal_folder = 'C:\\Users\\Sivar\\GitHub\\Breast_Cancer_Image_Segmentation\\data\\normal'

# Preprocess the images and masks from the benign, malignant, and normal folders
benign_images, benign_masks = preprocess_images_and_masks(benign_folder)
malignant_images, malignant_masks = preprocess_images_and_masks(malignant_folder)
normal_images, normal_masks = preprocess_images_and_masks(normal_folder)

# Concatenate the images and masks from the three folders
images = np.concatenate((benign_images, malignant_images, normal_images), axis=0)
masks = np.concatenate((benign_masks, malignant_masks, normal_masks), axis=0)

In [55]:
# Check the dimensions of the images and masks arrays
if len(images) == 0 or len(masks) == 0:
  print("Error: Images or masks array is empty")
elif len(images) != len(masks):
  print("Error: Mismatch in number of images and masks")
else:
  # Shuffle the images and masks together
  combined = list(zip(images, masks))
  np.random.shuffle(combined)
  images, masks = zip(*combined)

  # Convert the images and masks to numpy arrays
  images = np.array(images)
  masks = np.array(masks)

  # Calculate the number of images in each split
  num_images = len(images)
  num_train = int(0.90 * num_images)
  num_val = int(0.075 * num_images)
  num_test = num_images - num_train - num_val

  # Split the images and masks into training, validation, and test sets
  train_images = images[:num_train]
  train_masks = masks[:num_train]
  val_images = images[num_train:num_train+num_val]
  val_masks = masks[num_train:num_train+num_val]
  test_images = images[num_train+num_val:]
  test_masks = masks[num_train+num_val:]



In [56]:
# Define the data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomHeight(factor=(0.1, 0.2)),
    tf.keras.layers.experimental.preprocessing.RandomWidth(factor=(0.1, 0.2))
])

# Create a dataset with the data augmentation pipeline
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
dataset = dataset.map(lambda x, y: (data_augmentation(x), y))

# Configure the dataset for training
dataset = dataset.shuffle(buffer_size=1024).batch(32).prefetch(1)

In [ ]:
import matplotlib.pyplot as plt

# Print the first batch of augmented images and masks
for x, y in dataset.take(1):
    for i in range(len(x)):
        print(f'Augmented image {i}:')
        plt.imshow(x[i])
        plt.show()
        print(f'Augmented mask {i}:')
        plt.imshow(y[i], cmap='gray')
        plt.show()

(Display of some image/mask outputs)

Augmented Image 0: 

<img src='./images/aug_img_0.png' style='width:500px;height:400px;'>

Augmented Mask 0:

<img src='./images/aug_mask_0.png' style='width:500px;height:400px;'> 

Augmented Image 1: 

<img src='./images/aug_img_1.png' style='width:500px;height:400px;'>  

Augmented Mask 1:

<img src='./images/aug_mask_1.png' style='width:500px;height:400px;'> 

In [58]:
import tensorflow as tf
from keras.layers import Conv2DTranspose, BatchNormalization, ReLU, Conv2D, UpSampling2D, MaxPool2D, Dropout
from keras import Input, Model

def create_upsample_layer(filters, kernel_size, padding='same', kernel_initializer='he_normal'):
    """ This function creates a layer that upsamples an input tensor using a convolutional, batch
    normalization, and ReLU activation, followed by an upsampling operation. """
    def layer(x):
        x = Conv2D(filters, kernel_size, padding=padding, kernel_initializer=kernel_initializer)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = UpSampling2D(size=(2, 2))(x)
        return x
    return layer

def create_downsample_layer(filters, kernel_size, padding='same', kernel_initializer='he_normal'):
    """ This function creates a layer that downsamples an input tensor using a convolutional, batch
    normalization, and ReLU activation, followed by a max pooling operation. """
    def layer(x):
        x = Conv2D(filters, kernel_size, padding=padding, kernel_initializer=kernel_initializer)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = MaxPool2D(pool_size=(2, 2))(x)
        return x
    return layer

dropout_rate = 0.15

# Example usage
inputs = Input(shape=(256, 256, 3))
# Encoder part of the DeepUNet
x = create_downsample_layer(64, 3, padding='same', kernel_initializer='he_normal')(inputs)
x = create_downsample_layer(128, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_downsample_layer(256, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_downsample_layer(512, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_downsample_layer(1024, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_downsample_layer(2048, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)

# Decoder part of the DeepUNet
x = create_upsample_layer(1024, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_upsample_layer(512, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_upsample_layer(256, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_upsample_layer(128, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_upsample_layer(64, 3, padding='same', kernel_initializer='he_normal')(x)
x = Dropout(dropout_rate)(x)
x = create_upsample_layer(32, 3, padding='same', kernel_initializer='he_normal')(x)
outputs = Conv2D(1, 1, padding = 'same', activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)


In [ ]:
model.summary()


Model: "model_3"

| Layer (type)  |  Output Shape   | Param # |
|----------|----------|----------|
| input_4 (InputLayer)  | [(None, 256, 256, 3)]  | 0   |
| conv2d_39 (Conv2D)  | (None, 256, 256, 64)   | 1792   |
| batch_normalization_36 (BatchNormalization)    | (None, 256, 256, 64)   | 256   |
| re_lu_36 (ReLU)    | (None, 256, 256, 64)   | 0   |
| max_pooling2d_18 (MaxPooling2D)   | (None, 128, 128, 64)   | 0   |
| conv2d_40 (Conv2D)   | (None, 128, 128, 128)   | 73856   |
| batch_normalization_37 (BatchNormalization)  | (None, 128, 128, 128)   |  512   |
| re_lu_37 (ReLU)   | (None, 128, 128, 128)   | 0  |
| max_pooling2d_19  (MaxPooling2D)  | (None, 64, 64, 128)   | 0   |
| ...   |    |    |
| conv2d_42 (Conv2D)   | (None, 32, 32, 512)   | 1180160   |
| batch_normalization_39 (BatchNormalization)  | (None, 32, 32, 512)   | 2048   |


In [60]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define early stopping and reduce learning rate on plateau callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, 
                              verbose=1, mode='auto')

In [61]:
def dice_loss(y_true, y_pred):
    # Flatten the predictions and ground truth
    y_true_flat = tf.reshape(y_true, [-1])
    y_pred_flat = tf.reshape(y_pred, [-1])
    
    # Compute the intersection and union
    intersection = tf.reduce_sum(y_true_flat * y_pred_flat)
    union = tf.reduce_sum(y_true_flat) + tf.reduce_sum(y_pred_flat)
    
    # Compute the Dice loss
    dice_loss = 1 - 2 * intersection / union
    
    return dice_loss

# Compile the model with the Dice loss
model.compile(loss=dice_loss, optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the number of epochs and the batch size
num_epochs = 50
batch_size = 16

# Train the UNet model on the training data
history = model.fit(dataset, 
                    batch_size=batch_size, epochs=num_epochs,
                    callbacks=[early_stop, reduce_lr],
                    validation_data=(val_images, val_masks))


Epoch 1/50 \
22/22 [==============================] - 300s 13s/step - loss: 0.7448 - accuracy: 0.7145 - val_loss: 0.8166 - val_accuracy: 0.1092 - lr: 0.0010 \
Epoch 2/50 \
22/22 [==============================] - 264s 12s/step - loss: 0.6747 - accuracy: 0.8321 - val_loss: 0.8075 - val_accuracy: 0.1496 - lr: 0.0010 \
Epoch 3/50 \
22/22 [==============================] - 261s 12s/step - loss: 0.6309 - accuracy: 0.8832 - val_loss: 0.7907 - val_accuracy: 0.2538 - lr: 0.0010 \
Epoch 4/50 \
22/22 [==============================] - 260s 12s/step - loss: 0.5814 - accuracy: 0.9065 - val_loss: 0.6702 - val_accuracy: 0.7285 - lr: 0.0010 \
Epoch 5/50 \
22/22 [==============================] - 265s 12s/step - loss: 0.5450 - accuracy: 0.9211 - val_loss: 0.6946 - val_accuracy: 0.6409 - lr: 0.0010 \
Epoch 6/50 \
22/22 [==============================] - 265s 12s/step - loss: 0.4818 - accuracy: 0.9317 - val_loss: 0.5916 - val_accuracy: 0.7956 - lr: 0.0010 \
Epoch 7/50 \
22/22 [==============================] - 261s 12s/step - loss: 0.4092 - accuracy: 0.9476 - val_loss: 0.5503 - val_accuracy: 0.9095 - lr: 0.0010 \
... \
Epoch 49/50 \
22/22 [==============================] - 240s 11s/step - loss: 0.0918 - accuracy: 0.9875 - val_loss: 0.3449 - val_accuracy: 0.9462 - lr: 0.0010 \
Epoch 50/50 \
22/22 [==============================] - 252s 11s/step - loss: 0.0841 - accuracy: 0.9879 - val_loss: 0.2644 - val_accuracy: 0.9505 - lr: 0.0010 \

In [ ]:
# Epochs and corresponding val_loss and val_accuracy values
epochs_list = list(range(1, len(history.history['val_loss']) + 1))
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Plot the epoch vs val_loss
ax1 = plt.subplot(111)
ax1.plot(epochs_list, val_accuracy, 'r-')
ax1.set_ylabel('Validation Accuracy', color='r')

# Create a second y-axis for the val_accuracy
ax2 = ax1.twinx()
ax2.plot(epochs_list, val_loss, 'b-')
ax2.set_ylabel('Validation Loss', color='b')

# Set the same scaling on the y-axes
ax1.set_ylim([0.1, 1])
ax2.set_ylim([0.1, 1])

plt.show()


<img src='./images/loss_acc.png' style='width:600px;height:400px;'>  

In [ ]:
for i in range(5,10):
    # Select an image and its true mask
    image = test_images[i]
    mask = test_masks[i]

    # Make a prediction using the model
    prediction = model.predict(image[None, ...])[0]

    # Display the image and the true mask
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(image)
    ax1.set_title("Image")
    ax2.imshow(mask)
    ax2.set_title("True Mask")

    # Display the image and the model's prediction
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(image)
    ax1.set_title("Image")
    ax2.imshow(prediction)
    ax2.set_title("Model Mask")

    plt.show()


(Display of some image/mask outputs)

<img src='./images/true_mask_0.png' style='width:600px;height:300px;'> 

<img src='./images/model_mask_0.png' style='width:600px;height:300px;'> 

<img src='./images/true_mask_1.png' style='width:600px;height:300px;'> 

<img src='./images/model_mask_1.png' style='width:600px;height:300px;'>

<img src='./images/true_mask_2.png' style='width:600px;height:300px;'> 

<img src='./images/model_mask_2.png' style='width:600px;height:300px;'> 

<img src='./images/true_mask_3.png' style='width:600px;height:300px;'> 

<img src='./images/model_mask_3.png' style='width:600px;height:300px;'>  

There are a few reasons why the model might perform poorly on images without cancer. One reason is that it simply hasn't been trained on enough examples of these types of images. This means that the model doesn't have enough information to accurately classify these images. Another reason could be that the model is overfitting to the images that do contain cancer, and is therefore not able to generalize well to images without cancer.

One way to improve the performance on images without cancer would be to set a lower bound on the image segmentation. This would help the model to more accurately identify areas of the image that are likely to contain cancer, and would therefore improve its performance on images without cancer.

This project could be revisited in the future to set a lower bound on the image segmentation and add more non-cancerous images/masks to see if it leads to improved performance on images without cancer.

In [ ]:
# Make predictions on the test images
predictions = model.predict(test_images)

# Compute the mean IoU metric on the test set
mean_iou = tf.keras.metrics.MeanIoU(num_classes=2)
mean_iou.update_state(test_masks, predictions)
print("Mean IoU on test set: {:.3f}".format(mean_iou.result()))

# Compute the precision and recall on the test set
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

precision.update_state(test_masks, predictions)
recall.update_state(test_masks, predictions)

# Compute the F1 score on the test set
f1_score = 2 * (precision.result() * recall.result()) / (precision.result() + recall.result())
print("F1 score on test set: {:.3f}".format(f1_score))

# Reset the metrics
mean_iou.reset_states()
precision.reset_states()
recall.reset_states()


1/1 [==============================] - 1s 1s/step \
Mean IoU on test set: 0.735 \
F1 score on test set: 0.793